# COVID-19 Veri Keşfi ve Analizi

Bu notebook, COVID-19 verilerini keşfetmek ve temel analizler yapmak için tasarlanmıştır.

## 📊 İçerik
1. **Veri Yükleme**: API'lerden veri çekme
2. **Veri İnceleme**: Veri yapısı ve kalitesi analizi
3. **Temel İstatistikler**: Özet istatistikler
4. **Görselleştirmeler**: Temel grafikler
5. **Ülke Analizleri**: Türkiye ve diğer ülkeler

In [ ]:
# Gerekli kütüphaneleri import edelim
import sys
import os

# Proje modüllerimizi import edebilmek için path ekleyelim
project_root = os.path.abspath('..')
sys.path.append(project_root)
sys.path.append(os.path.join(project_root, 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Kendi modüllerimiz
from data_processing import CovidDataLoader, calculate_derived_metrics, calculate_time_series_metrics
from analysis import CovidAnalyzer
from visualization import CovidVisualizer

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("📦 Tüm kütüphaneler başarıyla yüklendi!")
print(f"🕒 Notebook başlatma zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. 📥 Veri Yükleme

COVID-19 verilerini farklı kaynaklardan yükleyeceğiz.

In [ ]:
# Data loader'ı başlat
print("🔄 COVID-19 veri yükleme işlemi başlıyor...")

loader = CovidDataLoader()

# Mevcut durum verilerini yükle (Disease.sh API)
print("\n🌍 Global mevcut durum verisi yükleniyor...")
snapshot_data = loader.load_disease_sh_data()

if snapshot_data is not None:
    print(f"✅ {len(snapshot_data)} ülkenin verisi yüklendi")
    print(f"📊 Sütunlar: {', '.join(snapshot_data.columns[:8])}...")
    
    # Türetilmiş metrikleri hesapla
    snapshot_data = calculate_derived_metrics(snapshot_data)
    print("✅ Türetilmiş metrikler eklendi")
else:
    print("❌ Snapshot verisi yüklenemedi")

print("\n" + "="*60)

In [ ]:
# Zaman serisi verilerini yükle (Johns Hopkins)
print("⏰ Zaman serisi verileri yükleniyor...")

# Confirmed cases
print("\n🔍 Confirmed cases verisi...")
confirmed_data = loader.load_johns_hopkins_data('confirmed')

if confirmed_data is not None:
    print(f"✅ {len(confirmed_data)} kayıt yüklendi")
    print(f"📅 Tarih aralığı: {confirmed_data['date'].min()} - {confirmed_data['date'].max()}")
    
    # Zaman serisi metriklerini hesapla
    confirmed_data = calculate_time_series_metrics(confirmed_data, 'confirmed')
    print("✅ Zaman serisi metrikleri eklendi")
else:
    print("❌ Confirmed verisi yüklenemedi")

print("\n" + "="*60)

## 2. 🔍 Veri İnceleme

Yüklediğimiz verilerin yapısını ve kalitesini inceleyelim.

In [ ]:
# Snapshot verisi inceleme
if snapshot_data is not None:
    print("🔍 SNAPSHOT VERİSİ ANALİZİ")
    print("=" * 40)
    
    print(f"📊 Boyut: {snapshot_data.shape}")
    print(f"📋 Sütun sayısı: {len(snapshot_data.columns)}")
    print(f"🌍 Ülke sayısı: {len(snapshot_data)}")
    
    # Veri türleri
    print("\n📈 Veri Türleri:")
    print(snapshot_data.dtypes.value_counts())
    
    # Eksik değerler
    print("\n❓ Eksik Değerler:")
    missing_data = snapshot_data.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
    if len(missing_data) > 0:
        for col, count in missing_data.head(10).items():
            print(f"   {col}: {count} ({count/len(snapshot_data)*100:.1f}%)")
    else:
        print("   ✅ Eksik değer yok")
    
    # Temel istatistikler
    print("\n📊 Temel İstatistikler (İlk 5 sütun):")
    numeric_cols = snapshot_data.select_dtypes(include=[np.number]).columns[:5]
    print(snapshot_data[numeric_cols].describe())

In [ ]:
# Zaman serisi verisi inceleme
if confirmed_data is not None:
    print("\n🔍 ZAMAN SERİSİ VERİSİ ANALİZİ")
    print("=" * 40)
    
    print(f"📊 Boyut: {confirmed_data.shape}")
    print(f"🌍 Ülke sayısı: {confirmed_data['country'].nunique()}")
    print(f"📅 Gün sayısı: {confirmed_data['date'].nunique()}")
    
    # Tarih aralığı detayları
    date_range = confirmed_data['date'].max() - confirmed_data['date'].min()
    print(f"⏱️  Toplam gün: {date_range.days}")
    
    # En fazla verisi olan ülkeler
    print("\n🏆 En Fazla Veri Noktası Olan Ülkeler:")
    country_counts = confirmed_data['country'].value_counts().head(10)
    for country, count in country_counts.items():
        print(f"   {country}: {count} gün")
    
    # Son tarih verisi özeti
    latest_date = confirmed_data['date'].max()
    latest_data = confirmed_data[confirmed_data['date'] == latest_date]
    print(f"\n📅 Son Tarih ({latest_date.strftime('%Y-%m-%d')}) Özeti:")
    print(f"   Toplam Global Vaka: {latest_data['confirmed'].sum():,}")
    print(f"   Ortalama Ülke Başına: {latest_data['confirmed'].mean():,.0f}")

## 3. 📈 Temel İstatistikler ve Global Analiz

In [ ]:
# Analiz sınıfını başlat
analyzer = CovidAnalyzer()

if snapshot_data is not None:
    # Global istatistikleri hesapla
    print("🌍 GLOBAL COVID-19 İSTATİSTİKLERİ")
    print("=" * 45)
    
    global_stats = analyzer.calculate_global_stats(snapshot_data)
    
    print(f"🏢 Toplam Ülke Sayısı: {global_stats['total_countries']:,}")
    print(f"🦠 Toplam Vaka: {global_stats['total_cases']:,}")
    print(f"💀 Toplam Ölüm: {global_stats['total_deaths']:,}")
    print(f"💚 Toplam İyileşen: {global_stats['total_recovered']:,}")
    print(f"🟡 Toplam Aktif: {global_stats['total_active']:,}")
    print(f"📊 Global CFR: {global_stats['global_cfr']:.2f}%")
    print(f"📈 Global İyileşme Oranı: {global_stats['global_recovery_rate']:.2f}%")
    print(f"🆕 Yeni Vakası Olan Ülke: {global_stats['countries_with_new_cases']}")
    print(f"⚰️  Yeni Ölümü Olan Ülke: {global_stats['countries_with_new_deaths']}")
    
    print("\n" + "="*60)

## 4. 🇹🇷 Türkiye Detaylı Analizi

In [ ]:
# Türkiye verilerini detaylı incele
if snapshot_data is not None:
    turkey_data = snapshot_data[snapshot_data['country'] == 'Turkey']
    
    if not turkey_data.empty:
        print("🇹🇷 TÜRKİYE COVID-19 DURUMU")
        print("=" * 35)
        
        turkey = turkey_data.iloc[0]
        
        print(f"📊 Toplam Vaka: {turkey.get('total_cases', 0):,}")
        print(f"💀 Toplam Ölüm: {turkey.get('total_deaths', 0):,}")
        print(f"💚 Toplam İyileşen: {turkey.get('total_recovered', 0):,}")
        print(f"🟡 Aktif Vaka: {turkey.get('active_cases', 0):,}")
        print(f"🆕 Yeni Vaka: {turkey.get('new_cases', 0):,}")
        print(f"⚰️  Yeni Ölüm: {turkey.get('new_deaths', 0):,}")
        
        print(f"\n📈 Vaka Ölüm Oranı (CFR): {turkey.get('case_fatality_rate', 0):.2f}%")
        print(f"📈 İyileşme Oranı: {turkey.get('recovery_rate', 0):.2f}%")
        print(f"📊 Milyon Başına Vaka: {turkey.get('cases_per_million', 0):,}")
        print(f"📊 Milyon Başına Ölüm: {turkey.get('deaths_per_million', 0):,}")
        
        # Şiddet indeksi hesapla
        severity_data = analyzer.calculate_severity_index(snapshot_data)
        turkey_severity = severity_data[severity_data['country'] == 'Turkey']
        if not turkey_severity.empty:
            severity_index = turkey_severity.iloc[0]['severity_index']
            severity_category = turkey_severity.iloc[0]['severity_category']
            print(f"⚠️  Şiddet İndeksi: {severity_index:.1f}/100 ({severity_category})")
    else:
        print("❌ Türkiye verisi bulunamadı")
    
    print("\n" + "="*60)

## 5. 🏆 En Çok Etkilenen Ülkeler

In [ ]:
# En çok etkilenen ülkeleri listele
if snapshot_data is not None:
    print("🏆 EN ÇOK ETKİLENEN ÜLKELER")
    print("=" * 35)
    
    # Toplam vaka bazında
    print("\n📊 Toplam Vaka Bazında (Top 10):")
    top_cases = snapshot_data.nlargest(10, 'total_cases')[['country', 'total_cases', 'total_deaths', 'cases_per_million']]
    
    for idx, row in top_cases.iterrows():
        print(f"{row.name+1:2d}. {row['country']:<20} - {row['total_cases']:>12,} vaka ({row['cases_per_million']:>8,.0f}/1M)")
    
    # Milyon başına vaka bazında
    print("\n📈 Milyon Başına Vaka Bazında (Top 10):")
    top_per_million = snapshot_data.nlargest(10, 'cases_per_million')[['country', 'total_cases', 'cases_per_million', 'population']]
    
    for idx, row in top_per_million.iterrows():
        pop_millions = row.get('population', 0) / 1000000 if row.get('population', 0) > 0 else 0
        print(f"{row.name+1:2d}. {row['country']:<20} - {row['cases_per_million']:>8,.0f}/1M (Nüfus: {pop_millions:.1f}M)")
    
    print("\n" + "="*60)

## 6. 📊 Temel Görselleştirmeler

Verileri görsel olarak inceleyeceğiz.

In [ ]:
# Görselleştirme sınıfını başlat
visualizer = CovidVisualizer()

if snapshot_data is not None:
    print("📊 Görselleştirmeler oluşturuluyor...")
    
    # 1. En çok etkilenen ülkeler (toplam vaka)
    print("\n📈 Top 15 ülke - Toplam vaka grafiği oluşturuluyor...")
    fig1 = visualizer.plot_top_countries(snapshot_data, 'total_cases', top_n=15)
    if fig1:
        plt.tight_layout()
        plt.show()
        plt.close(fig1)
    
    print("✅ İlk grafik tamamlandı")

In [ ]:
# Türkiye detay grafiği
if snapshot_data is not None:
    print("🇹🇷 Türkiye özet grafiği oluşturuluyor...")
    fig2 = visualizer.plot_country_summary(snapshot_data, 'Turkey')
    if fig2:
        plt.tight_layout()
        plt.show()
        plt.close(fig2)
    
    print("✅ Türkiye grafiği tamamlandı")

In [ ]:
# Korelasyon analizi
if snapshot_data is not None:
    print("🔍 Korelasyon matrisi oluşturuluyor...")
    fig3 = visualizer.plot_correlation_matrix(snapshot_data)
    if fig3:
        plt.tight_layout()
        plt.show()
        plt.close(fig3)
    
    print("✅ Korelasyon matrisi tamamlandı")

## 7. ⏰ Zaman Serisi Analizi

In [ ]:
# Zaman serisi görselleştirme
if confirmed_data is not None:
    print("⏰ Zaman serisi analizi başlıyor...")
    
    # Türkiye, ABD, Almanya karşılaştırması
    countries_to_compare = ['Turkey', 'United States', 'Germany', 'Italy', 'Spain']
    
    print(f"\n📊 {len(countries_to_compare)} ülke karşılaştırması grafik oluşturuluyor...")
    fig4 = visualizer.plot_time_series(confirmed_data, countries_to_compare, 'confirmed')
    if fig4:
        plt.tight_layout()
        plt.show()
        plt.close(fig4)
    
    print("✅ Zaman serisi grafiği tamamlandı")

## 8. 📋 Ülke Karşılaştırmaları

In [ ]:
# Ülke karşılaştırması tablosu
if snapshot_data is not None:
    print("🏆 ÜLKE KARŞILAŞTIRMASI")
    print("=" * 25)
    
    # Karşılaştırılacak ülkeler
    comparison_countries = ['Turkey', 'Germany', 'United States', 'Italy', 'Spain', 'France']
    comparison_metrics = ['total_cases', 'total_deaths', 'cases_per_million', 'deaths_per_million']
    
    comparison_df = analyzer.compare_countries(snapshot_data, comparison_countries, comparison_metrics)
    
    if not comparison_df.empty:
        print("\n📊 Karşılaştırma Tablosu:")
        print("-" * 100)
        
        # Güzel formatlı tablo
        for idx, row in comparison_df.iterrows():
            country = row['country']
            cases = f"{row.get('total_cases', 0):>12,}"
            deaths = f"{row.get('total_deaths', 0):>10,}"
            cases_pm = f"{row.get('cases_per_million', 0):>10,.0f}"
            deaths_pm = f"{row.get('deaths_per_million', 0):>10,.0f}"
            
            print(f"{country:<15} | {cases} vaka | {deaths} ölüm | {cases_pm}/1M | {deaths_pm}/1M")
        
        print("-" * 100)
        
        # Sıralama bilgileri
        print("\n🏅 Sıralamalar (Toplam Vaka):")
        for idx, row in comparison_df.sort_values('total_cases', ascending=False).iterrows():
            rank = int(row.get('total_cases_rank', 0))
            print(f"   {rank}. {row['country']}: {row.get('total_cases', 0):,} vaka")
    
    print("\n" + "="*60)

## 9. 📝 Özet ve Sonuçlar

In [ ]:
# Genel özet
print("📝 VERİ KEŞFİ SONUÇLARI")
print("=" * 30)

if snapshot_data is not None:
    print(f"✅ Başarıyla {len(snapshot_data)} ülkenin verisi analiz edildi")
    
    # Veri kalitesi özeti
    total_cases = snapshot_data['total_cases'].sum()
    total_deaths = snapshot_data['total_deaths'].sum()
    countries_with_data = len(snapshot_data[snapshot_data['total_cases'] > 0])
    
    print(f"📊 Global toplam vaka: {total_cases:,}")
    print(f"💀 Global toplam ölüm: {total_deaths:,}")
    print(f"🌍 Veri bulunan ülke: {countries_with_data}")
    
    if confirmed_data is not None:
        print(f"📅 Zaman serisi veri noktası: {len(confirmed_data):,}")
        date_range = confirmed_data['date'].max() - confirmed_data['date'].min()
        print(f"⏱️  Zaman aralığı: {date_range.days} gün")

print("\n🎯 Sonraki Adımlar:")
print("   1. Daha detaylı istatistiksel analizler")
print("   2. Makine öğrenmesi modelleri")
print("   3. İnteraktif dashboard geliştirme")
print("   4. Tahmin modellerinin oluşturulması")

print(f"\n🕒 Analiz tamamlanma zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🎉 Veri keşfi başarıyla tamamlandı!")